In [194]:
import json, ast,time
import pandas as pd

from utils import api, db, prompt, prompting

from importlib import reload

from dotenv import load_dotenv
load_dotenv()

True

In [183]:
reload(prompt)
reload(api)
reload(db)
reload(prompting)

<module 'utils.prompting' from '/Volumes/Personal/Work/Feedloop/fact_ext/from_github/AI-Research/utils/prompting.py'>

In [46]:
total_prompt_token = 0
total_completion_token = 0

# Style Transfer

In [47]:
def run_model(eval_prompt):
    return api.get_completions_gpt4_turbo([
        {
        "role": "user",
        "content": eval_prompt,
        }
    ], temp=0.8)

In [48]:
def increase_token_usage(completion):
    global total_prompt_token, total_completion_token
    total_prompt_token+=completion.usage.prompt_tokens
    total_completion_token+=completion.usage.completion_tokens

In [49]:
def convert_to_json(response):
    try:
        res_json = ast.literal_eval(response)
    except:
        res_json = ast.literal_eval(response.replace("null", "None"))
    return res_json

In [50]:
def run_prompt(text):
    eval_prompt = prompt.style_transfer4(text)
    eval_completion = run_model(eval_prompt)
    
    increase_token_usage(eval_completion)

    return convert_to_json(eval_completion.choices[0].message.content)

In [22]:
def update_result(new_res):
    # formals = formal.split(". ")
    formals = new_res["formal"].split(". ")
    informals = new_res["informal"].split(". ")
    alays = new_res["alay"].split(". ")
    jaksels = new_res["jaksel"].split(". ")
    res_list = []
    for i in range(5):
        new_dict = {}
        new_dict["formal"] = formals[i]
        new_dict["informal"] = informals[i]
        new_dict["alay"] = alays[i]
        new_dict["jaksel"] = jaksels[i]
        res_list.append(new_dict)
    return res_list

In [21]:
def update_result1(new_res):
    res_list = []
    for i in range(5):
        new_dict = {}
        new_dict["formal"] = new_res["formal"][i]
        new_dict["informal"] = new_res["informal"][i]
        new_dict["alay"] = new_res["alay"][i]
        new_dict["jaksel"] = new_res["jaksel"][i]
        res_list.append(new_dict)
    return res_list

In [23]:
def save_result(result):
    # res_list = update_result(result)
    res_list = update_result1(result)
    with open(f'./style_transfer/test_final.json', 'r') as json_file:
        dict_temp = json.load(json_file)
    dict_temp["result"].extend(res_list)
    with open("./style_transfer/test_final.json", 'w') as json_file:
        json.dump(dict_temp, json_file, indent=2)
    print(len(dict_temp["result"]))

In [10]:
text = """Saya ingin memimpin, jadi saya berjalan di depan, saudara laki-laki saya mengikuti saya, dan ayah saya di belakang. Itu adalah hari yang indah ketika kami mulai berjalan; langit biru tua di atas kami, matahari menyinari kami dari atas kepala kami, dan di sekeliling Anda bisa mendengar dengungan serangga dan gemerisik angin di dedaunan semak hijau yang indah. Aku bisa mencium bau indah bunga musim panas di sekitarku. Tiba-tiba seekor ular besar muncul di hadapan saya. Awalnya, saya tidak peduli dan mengganggunya dengan kayu kecil. Namun, ular tersebut langsung mengejar saya dan berusaha menggigit. Saya panik dan berlari sekencang mungkin. Ayah yang melihat peristiwa tersebut langsung memanggil nama saya secara histeris dan menarik saya sekuat tenaga dari area hutan. Meski sudah bertahun-tahun, cerita ini masih membekas di benak saya. Saya sangat ingat hari itu, dan saya bersyukur ayah saya menyelamatkan saya dari gigitan ular. Saya juga belajar bahwa saya harus lebih berhati-hati saat berjalan di alam liar dan memperlakukan hewan dengan hormat. Itulah mengapa ini adalah peristiwa penting dalam hidup saya."""

In [38]:
themes = ["Indonesian Culture and Traditions", "Indonesian Cuisine", "Tourism in Indonesia", "Indonesian History", "Environmental Issues in Indonesia", "Education System in Indonesia","Learning Japanese Language", "Understanding Japanese Culture", "Travelling in Japanese", "Cooking Japanese Dishes", "Hobbies", "Experience as a Teacher", "Mohammad Hatta", "Raden Ajeng Kartini", "Pramoedya Ananta Toer", "B.J. Habibie", "Susi Pudjiastuti", "Dian Sastro", "Nicholas Saputra","Retno Marsudi", "Anies Baswedan", "Ganjar Pranowo", "Ridwan Kamil","Malin Kundang", "Roro Jonggrang", "Nyi Roro Kidul", "Keong Mas", "The Tale of Cendrawasih","Developments in Renewable Energy Technologies", "Global Health Challenges and Medical Breakthroughs", "Trends and Innovations in the Automotive Industry", "Impact of Social Media on Society and Culture", "Major Sporting Events and Athlete Achievements","Healthy Recipe Ideas for a Balanced Diet","Tips for Sustainable Living and Eco-Friendly Practices","Travel Destinations: Hidden Gems Around the World",
"Home Workout Routines for Beginners",
"DIY Home Decor Projects and Ideas",
"Mindfulness and Mental Wellness Practices",
"Fashion Trends: Seasonal Styles and Accessories",
"Book Recommendations and Reviews",
"Pet Care Advice for New Pet Owners",
"Photography Tips for Amateur Photographers","Pride and Prejudice by Jane Austen",
"1984 by George Orwell",
# "To Kill a Mockingbird by Harper Lee",
# "The Great Gatsby by F. Scott Fitzgerald",
# "Moby-Dick by Herman Melville",
# "War and Peace by Leo Tolstoy",
# "Hamlet by William Shakespeare",
"One Hundred Years of Solitude by Gabriel García Márquez",
"The Odyssey by Homer",
"The Catcher in the Rye by J.D. Salinger","The Structure of DNA by James Watson and Francis Crick",
"Relativity: The Special and General Theory by Albert Einstein",
# "The Interpretation of Dreams by Sigmund Freud",
# "On the Electrodynamics of Moving Bodies by Albert Einstein",
# "The Wealth of Nations by Adam Smith",
# "The Origin of Species by Charles Darwin",
"Principia Mathematica by Isaac Newton",
"A Brief History of Time by Stephen Hawking",
"Silent Spring by Rachel Carson",
"The Social Contract by Jean-Jacques Rousseau","Apple iPhone 14",
"Bose QuietComfort 35 II Wireless Headphones",
"CeraVe Moisturizing Cream",
"Dell XPS 15 Laptop",
# "Philips Hue Smart Lighting System",
# "Seventh Generation Natural Laundry Detergent",
# "Fitbit Charge 5 Fitness Tracker",
# "LEGO Mindstorms Robot Inventor Building Set",
"Canon EOS 5D Mark IV DSLR Camera",
"HelloFresh Meal Kit Subscription Service"]

In [ ]:
themes = ["Indonesian Culture and Traditions", "Indonesian Cuisine", "Tourism in Indonesia", "Indonesian History", "Basic Science", "Technology", "Environmental Issues in Indonesia", "Health and Wellness", "Education System in Indonesia","Learning Japanese Language", "Understanding Japanese Culture", "Travelling in Japanese", "Cooking Japanese Dishes", "Education in Japanese","Hobbies", "Experience as a Teacher", "Soekarno", "Mohammad Hatta", "Soeharto", "Raden Ajeng Kartini", "Pramoedya Ananta Toer", "B.J. Habibie", "Susi Pudjiastuti", "Dian Sastro", "Nicholas Saputra","Retno Marsudi", "Anies Baswedan", "Ganjar Pranowo", "Ridwan Kamil","Malin Kundang", "Roro Jonggrang", "Sangkuriang", "Lutung Kasarung", "Nyi Roro Kidul", "Keong Mas", "The Tale of Cendrawasih","Political Elections and Leadership Changes Worldwide", "Developments in Renewable Energy Technologies", "Global Health Challenges and Medical Breakthroughs", "Trends and Innovations in the Automotive Industry", "Impact of Social Media on Society and Culture", "Major Sporting Events and Athlete Achievements","Healthy Recipe Ideas for a Balanced Diet","Tips for Sustainable Living and Eco-Friendly Practices","Travel Destinations: Hidden Gems Around the World",
"Home Workout Routines for Beginners",
"DIY Home Decor Projects and Ideas",
"Mindfulness and Mental Wellness Practices",
"Fashion Trends: Seasonal Styles and Accessories",
"Book Recommendations and Reviews",
"Pet Care Advice for New Pet Owners",
"Photography Tips for Amateur Photographers","Pride and Prejudice by Jane Austen",
"1984 by George Orwell",
# "To Kill a Mockingbird by Harper Lee",
# "The Great Gatsby by F. Scott Fitzgerald",
# "Moby-Dick by Herman Melville",
# "War and Peace by Leo Tolstoy",
# "Hamlet by William Shakespeare",
"One Hundred Years of Solitude by Gabriel García Márquez",
"The Odyssey by Homer",
"The Catcher in the Rye by J.D. Salinger","The Structure of DNA by James Watson and Francis Crick",
"Relativity: The Special and General Theory by Albert Einstein",
# "The Interpretation of Dreams by Sigmund Freud",
# "On the Electrodynamics of Moving Bodies by Albert Einstein",
# "The Wealth of Nations by Adam Smith",
# "The Origin of Species by Charles Darwin",
"Principia Mathematica by Isaac Newton",
"A Brief History of Time by Stephen Hawking",
"Silent Spring by Rachel Carson",
"The Social Contract by Jean-Jacques Rousseau","Apple iPhone 14",
"Bose QuietComfort 35 II Wireless Headphones",
"CeraVe Moisturizing Cream",
"Dell XPS 15 Laptop",
# "Philips Hue Smart Lighting System",
# "Seventh Generation Natural Laundry Detergent",
# "Fitbit Charge 5 Fitness Tracker",
# "LEGO Mindstorms Robot Inventor Building Set",
"Canon EOS 5D Mark IV DSLR Camera",
"HelloFresh Meal Kit Subscription Service"]

In [39]:
print(len(themes))

60


In [31]:
t = "1984 by George Orwell"
test_input = f"Generate a 5-sentences text conversation about {t}"
result = run_prompt(test_input)

In [40]:
for t in themes:
    # input = f"Generate a 5-sentences paragraph for {t} review"
    input = f"Generate a 5-sentences text conversation about {t}"
    # input = f"Generate 5 distinct questions about {t}"
    result = run_prompt(input)
    save_result(result)
    time.sleep(10)

928
933
938
943
948
953
958
963
968
973
978
983
988
993
998
1003
1008
1013
1018
1023
1028
1033
1038
1043
1048
1053
1058
1063
1068
1073
1078
1083
1088
1093
1098
1103
1108
1113
1118
1123
1128
1133
1138
1143
1148
1153
1158
1163
1168
1173
1178
1183
1188
1193
1198
1203
1208
1213
1218
1223


# Instruction Generation

In [ ]:
gen_instruction_prompt = """--- Reference Text:
Create the {{Indonesian}} translation from the given {{text}}.

--- Instruction:
Generate 25 variation texts from the reference text, each one should be distinct. Provide output based on JSON structure below:
{
    "gen_instructions": <generated texts>[]
}
"""
cur_res = run_prompt(gen_instruction_prompt)

In [168]:
def save_instruction(result):
    # res_list = update_result(result)
    res_list = result["gen_instructions"]
    with open(f'./style_transfer/instructions_tr.json', 'r') as json_file:
        dict_temp = json.load(json_file)
    dict_temp["instructions"].extend(res_list)
    with open("./style_transfer/instructions_tr.json", 'w') as json_file:
        json.dump(dict_temp, json_file, indent=2)
    print(len(dict_temp["instructions"]))

In [176]:
save_instruction(cur_res)

75


## Generate Result

In [160]:
def gen_instruction_json(source, target):
    with open(f'./style_transfer/ins_style.json', 'r') as json_file:
        dict_res = json.load(json_file)
    with open(f'./style_transfer/instructions.json', 'r') as json_file:
        dict_inst = json.load(json_file)
    with open(f'./style_transfer/style_trf_data.json', 'r') as json_file:
        dict_data = json.load(json_file)
    c = 0
    for i in dict_data["result"]:
        text = ""
        if c > 197:
            c = 0
        dict_res["output"].append(i[target])
        input = i[source]
        if (c) <= 102:
            text = f"Text: {input}\n\n"
            if c%2 != 0:
                text+="--- Instruction:\n"
            dict_res["input"].append(text+dict_inst["instructions"][c].format(target))
            c += 1
            continue
        if (c)%2 != 0:
            text+="--- Instruction:\n"
        text+=dict_inst["instructions"][c].format(target)
        dict_res["input"].append(text+f"\n\nText: {input}")
        c += 1
    with open("./style_transfer/ins_style.json", 'w') as json_file:
        json.dump(dict_res, json_file, indent=2)
    

In [210]:
def gen_instruction_json1(source, target):
    with open(f'./style_transfer/ins_trans.json', 'r') as json_file:
        dict_res = json.load(json_file)
    with open(f'./style_transfer/instructions_tr1.json', 'r') as json_file:
        dict_inst = json.load(json_file)
    df_trans = pd.read_csv("./style_transfer/train.csv")
    c = 0
    dict_ind = {"indonesian": "indonesia", "english": "inggris", "sundanese":"sunda", "javanese":"jawa"}
    target_ind = dict_ind[target]
    for _, row in df_trans.iterrows():
        text = ""
        if c > 49:
            c = 0
        dict_res["output"].append(row[target])
        input = row[source]
        if (c) <= 24:
            text = f"Text: {input}\n\n"
            if c%2 != 0:
                text+="--- Instruction:\n"
            dict_res["input"].append(text+dict_inst["instructions"][c].format(target_ind))
            c += 1
            continue
        if (c)%2 != 0:
            text+="--- Instruction:\n"
        text+=dict_inst["instructions"][c].format(target_ind)
        dict_res["input"].append(text+f"\n\nText: {input}")
        c += 1
    with open("./style_transfer/ins_trans.json", 'w') as json_file:
        json.dump(dict_res, json_file, indent=2)

In [211]:
lang_opt = ["indonesian", "english", "sundanese", "javanese"]
for i in lang_opt:
    for j in lang_opt:
        if j == i:
            continue
        gen_instruction_json1(i, j)

In [162]:
lang_styles = ["formal", "informal", "alay", "jaksel"]
for i in lang_styles:
    for j in lang_styles:
        if j == i:
            continue
        gen_instruction_json(i, j)

In [217]:
with open(f'./style_transfer/ins_trans.json', 'r') as json_file:
    dict_inst = json.load(json_file)
print(len(dict_inst["input"]))
print(dict_inst["input"][8888])
print(dict_inst["output"][8888])

1223


In [85]:
with open(f'./style_transfer/instructions.json', 'r') as json_file:
    dict_temp = json.load(json_file)
for i in range(len(dict_temp["instructions"])):
    if i == 102:
        print(dict_temp["instructions"][i])
        break
# for t in dict_temp["instructions"]:
#     print(t.format("alay"))

Proceed to translate the text into Indonesian, applying a {} tone throughout.


In [ ]:
with open(f'./style_transfer/instructions.json', 'r') as json_file:
    dict_ins = json.load(json_file)
for i in dict_ins["instructions"]:
    print(i.format("jaksel"))

In [180]:
print(f"prompt token: {total_prompt_token}\ncompletion token: {total_completion_token}")
prompt_token_price = total_prompt_token * 0.01 * 0.001
completion_token_price = total_completion_token * 0.03 * 0.001
print(f"prompt token price: {round(prompt_token_price,2)}\ncompletion token price: {round(completion_token_price,2)}")

prompt token: 2735
completion token: 8386
prompt token price: 0.03
completion token price: 0.25
